# Launching Walmart Lite - Analysis

Main Objective: Perform analysis on datasets to find optimal top 10 DMAs to launch a "Walmart-Lite" store in.

First, we'll import the necessary libraries and read the dataset files into DataFrames that Python/Pandas can use.

In [218]:
import datetime
import matplotlib.pyplot as plt
import ntpath
import numpy as np
import operator
import os
import pandas as pd
import statistics as stats
import time

from collections import OrderedDict
from pathlib import Path
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

In [219]:
#Purpose: Read in dataset files into pandas DataFrame
datasets = {}
files = []

def loadFile(path):
    global files
    global datasets
    
    fileToLoad = Path('datasets/norteast.xlsx')
    fileExists = fileToLoad.is_file()
    if not fileExists:
        return False
    
    files.append(path)
    
    filename,fileExtension = os.path.splitext(path)
    
    if '.csv' == fileExtension:
        df = pd.read_csv(path)
    elif '.xlsx' == fileExtension:
        df = pd.read_excel(path)
    else:
        return False
    
    datasets[ntpath.basename(path)] = df
    return True

def mapDMAToName(data,searchTerm,isExactMatch = True):
    if isExactMatch:
        result = data.query("DMA == @searchTerm")
        if len(result) >= 1:
            return result.values[0][1]
        else:
            return False
        
def mapStoreNumberToDMA(storeNumber):
    result = datasets["stores data-set-withDMA.xlsx"].query("Store == @storeNumber")
    if len(result) >= 1:
        return result.values[0][3]
    else:
        return False
    
def mapDMAToStoreNumber(DMA):
    result = datasets["stores data-set-withDMA.xlsx"].iterrows()
    if len(result) >= 1:
        return result.values[0][0]
    else:
        return False
    
def rankDictionary(dictionaryToRank):
    sorted_x = sorted(dictionaryToRank.items(), key=lambda kv: kv[1])
    out_dict = OrderedDict(sorted_x)
    return out_dict

loadFile("datasets/modified/Features data set.csv")
loadFile("datasets/sales data-set.csv")
loadFile("datasets/stores data-set-withDMA.xlsx")

True

Given the restriction in the assignment specification, we need to adjust the timeline of the features dataset accordingly to only include February 5, 2010 to January 11, 2012.

In [220]:
#Purpose: Adjust features dataset timeline to restriction (Feburary 5, 2010 to January 11, 2012)

def adjustFeaturesTimeline(startDate,endDate):
    global datasets
    startDate = datetime.datetime.strptime(startDate, "%d/%m/%Y").date()
    endDate = datetime.datetime.strptime(endDate, "%d/%m/%Y").date()
    
    for index,row in datasets["Features data set.csv"].iterrows():
        currentDate = datetime.datetime.strptime(row['Date'], "%d/%m/%Y").date()
        if currentDate > endDate or currentDate < startDate:
            datasets["Features data set.csv"] = datasets["Features data set.csv"].drop([index])


adjustFeaturesTimeline("05/02/2010","01/11/2012")

The sheets containing consumer info for each area are structured differently than the features and sales sheets, so they will need to be imported differently.

In [221]:
#Purpose: Import consumer info per area for analysis (population count, income, expenditures related to retail)

usefulData = [
    'Item',
    'Number of consumer units (in thousands)',
    'Income before taxes',
    'Age of reference person',
    'Adults 65 and older',
    'Earners',
    'Vehicles',
    'Percent homeowner',
    'Average annual expenditures',
    'Food',
    'Alcoholic beverages',
    'Apparel and services',
    'Personal care products and services',
    'Tobacco products and smoking supplies'
]

areas = {}
areaInfo = {}

loadFile("datasets/norteast.xlsx")
loadFile("datasets/midwest.xlsx")
loadFile("datasets/south.xlsx")
loadFile("datasets/west.xlsx")

def loadMSA(fileName):
    global datasets
    global usefulData
    global areas
    
    bufferListOfPlaces = []
    bufferDict = {}
    for value in datasets[fileName].values:
        attribute = value[0]
        if attribute in usefulData:
            if(attribute == 'Item'): # List of DMAs
                bufferListOfPlaces = value[1:]
            else:
                bufferDict[attribute] = value
        
    if len(bufferListOfPlaces) > 0:
        i = 1
        for place in bufferListOfPlaces:
            place = place.replace('\n',' ')
            for _,value in bufferDict.items():
                attribute = value[0]
                attributeValue = value[i]

                if place not in areas:
                    areas[place] = {}
                
                areas[place][attribute] = attributeValue
            i+=1
    else:
        return False
    
                
loadMSA("midwest.xlsx")
loadMSA("norteast.xlsx")
loadMSA("south.xlsx")
loadMSA("west.xlsx")

In [222]:
#Purpose: Get a median for all sales in stores

loadFile("datasets/custom/custom-dma-to-consumerArea.csv")

sales = datasets["sales data-set.csv"].drop(['Dept','Date','IsHoliday'],axis=1)

def getSalesByStore():
    global sales
    
    storeSales = {}
    
    for _,row in sales.iterrows():
        storeNumber = int(row['Store'])
        weeklySales = row['Weekly_Sales']
        if storeNumber in storeSales:
            storeSales[storeNumber].append(weeklySales)
        else:
            storeSales[storeNumber] = [weeklySales]
    
    return storeSales

storeSales = getSalesByStore() # Very slow
    
DMASpendingPerCapita = {}
DMASpending = {}

for index,store in storeSales.items():
    DMA = mapStoreNumberToDMA(index)
    consumerInfoName = mapDMAToName(datasets["custom-dma-to-consumerArea.csv"],DMA)
    if type(consumerInfoName) != float:
        medianStoreSales = stats.median(storeSales[index])
        if DMA not in DMASpendingPerCapita:
            DMASpendingPerCapita[DMA] = []
            DMASpending[DMA] = []

        DMASpendingPerCapita[DMA].append(medianStoreSales)

for index,listOfSales in DMASpendingPerCapita.items():
    sum = 0
    for sale in listOfSales:
        sum+=sale
    
    consumerInfoName = mapDMAToName(datasets["custom-dma-to-consumerArea.csv"],index)
    DMASpending[index] = sum
    DMASpendingPerCapita[index] = sum/areas[consumerInfoName]['Number of consumer units (in thousands)']


print("Rankings of Overall Spending (Sum of Medians of Stores)")
for index,value in rankDictionary(DMASpending).items():
    print(index + " : " + str(value))
print()
print("Rankings of Spending per Capita (Sum of Medians of Stores/Number of Customer Units)")
for index,value in rankDictionary(DMASpendingPerCapita).items():
    print(index + " : " + str(value))

Rankings of Overall Spending (Sum of Medians of Stores)
San Diego : 8896.69
Denver : 15188.0
Atlanta : 20756.095
Tampa-St. Pete : 22269.079999999998
Chicago : 23870.585
Philadelphia : 24403.07
Dallas-Ft.Worth : 34491.115000000005
Cleveland-Akron : 39530.765
Houston : 47302.36
Los Angeles : 53975.275

Rankings of Spending per Capita (Sum of Medians of Stores/Number of Customer Units)
Chicago : 6.367187249933315
San Diego : 7.151680064308682
Los Angeles : 8.517480669086318
Philadelphia : 10.013569963069347
Atlanta : 10.55752543234995
Denver : 10.680731364275669
Dallas-Ft.Worth : 12.474182640144667
Tampa-St. Pete : 18.573044203502917
Houston : 18.674441373864983
Cleveland-Akron : 37.576772813688216


Since there is no truth data for which market is the best, we will turn to K-Means, an unsupervised machine learning model to separate our features dataset into clusters.

In [223]:
clustersToTry = [2,3,4]

featuresSetForAnalysis = datasets["Features data set.csv"].drop(['IsHoliday','Date'],axis=1)

def runKMeans(data,numOfClusters):
    kmeans = KMeans(n_clusters=numOfClusters)

    kmeans.fit(data)
    y_km = kmeans.fit_predict(data)
    return y_km

for clusterNum in clustersToTry:
    exportPath = 'export\Features data set.csv\\' + str(clusterNum) + '.csv'
    datasetToExport = datasets["Features data set.csv"].assign(cluster = runKMeans(featuresSetForAnalysis,clusterNum).tolist())
    datasetToExport.to_csv(exportPath,index=False)